# 5.3 Lab: Cross-Validation and the Bootstrap

## 5.3.1 The Validation Set Approach

In [1]:
# load the ISLR library
library(ISLR)

# get the Auto dataset
data(Auto)
auto = Auto

# set the seed for reproducibility
set.seed(1)

In [2]:
# recalling that the Auto dataset has 392 observations split the dataset in half

train_idx = sample(392, 196)

train_data = auto[train_idx, ]
test_data = auto[-train_idx, ]

In [3]:
# train some models
lm_1 = lm(mpg ~ horsepower, data=train_data)
lm_2 = lm(mpg ~ poly(horsepower, 2), data=train_data)
lm_3 = lm(mpg ~ poly(horsepower, 3), data=train_data)

In [4]:
# write a function to calculate the MSE
calc_MSE = function (model, data){
    return (
        mean((data$mpg - predict(model, data)) ^ 2)
    )
}

In [5]:
# calculate the train MSE for the three models
calc_MSE(lm_1, train_data)
calc_MSE(lm_2, train_data)
calc_MSE(lm_3, train_data)

[1] 21.78987

[1] 18.29115

[1] 18.27482

In [6]:
# calculate the train MSE for the three models
calc_MSE(lm_1, test_data)
calc_MSE(lm_2, test_data)
calc_MSE(lm_3, test_data)

[1] 26.14142

[1] 19.82259

[1] 19.78252

In [7]:
# do the splitting, training and testing with a different seed

# set the seed for reproducibility
set.seed(2)

# recalling that the Auto dataset has 392 observations split the dataset in half

train_idx = sample(392, 196)

train_data = auto[train_idx, ]
test_data = auto[-train_idx, ]

# train more models
lm_1 = lm(mpg ~ horsepower, data=train_data)
lm_2 = lm(mpg ~ poly(horsepower, 2), data=train_data)
lm_3 = lm(mpg ~ poly(horsepower, 3), data=train_data)

# calculate the train MSE for the three models
calc_MSE(lm_1, train_data)
calc_MSE(lm_2, train_data)
calc_MSE(lm_3, train_data)

# calculate the train MSE for the three models
calc_MSE(lm_1, test_data)
calc_MSE(lm_2, test_data)
calc_MSE(lm_3, test_data)

[1] 25.02717

[1] 19.19628

[1] 19.13865

[1] 23.29559

[1] 18.90124

[1] 19.2574

## 5.3.2: Leave-One-Out Cross-Validation

In [8]:
# the the glm function has the same behavior as the lm function by default, but with expanded output
# we'll use the glm because cv.glm, the function in boot, works with glm objects

lm_1 = glm_1 = glm(mpg ~ horsepower, data=auto)
glm_1 = glm(mpg ~ horsepower, data=auto)

summary(lm_1)
summary(glm_1)


Call:
glm(formula = mpg ~ horsepower, data = auto)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-13.5710   -3.2592   -0.3435    2.7630   16.9240  

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 39.935861   0.717499   55.66   <2e-16 ***
horsepower  -0.157845   0.006446  -24.49   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 24.06645)

    Null deviance: 23819.0  on 391  degrees of freedom
Residual deviance:  9385.9  on 390  degrees of freedom
AIC: 2363.3

Number of Fisher Scoring iterations: 2



Call:
glm(formula = mpg ~ horsepower, data = auto)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-13.5710   -3.2592   -0.3435    2.7630   16.9240  

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 39.935861   0.717499   55.66   <2e-16 ***
horsepower  -0.157845   0.006446  -24.49   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 24.06645)

    Null deviance: 23819.0  on 391  degrees of freedom
Residual deviance:  9385.9  on 390  degrees of freedom
AIC: 2363.3

Number of Fisher Scoring iterations: 2


In [9]:
# load the boot library and cross-validate glm_1
library(boot)
cv_err = cv.glm(auto, glm_1)
cv_err$delta[1]

[1] 24.23151

In [10]:
# cross-validate the linear through quintic model of horsepower on mpg

#note: LOOCV is not effected by the seed

# create a vector of 5 0s to store the results
cv_errors = rep(0, 5)

for (i in 1:5) {
    glm_fit = glm(mpg ~ poly(horsepower, i), data=auto)
    cv_errors[i] = cv.glm(auto, glm_fit)$delta[1]
}

cv_errors

[1] 24.23151 19.24821 19.33498 19.42443 19.03321

## 5.3.3 k-Fold Cross-Validation

In [11]:
# cross-validate the linear through 10th degree model of horsepower on mpg

# set a seed
set.seed(17)

# create a vector of 10 0s to store the results
cv_errors = rep(0, 10)

for (i in 1:10) {
    glm_fit = glm(mpg ~ poly(horsepower, i), data=auto)
    cv_errors[i] = cv.glm(auto, glm_fit, , K=10)$delta[1]
}

cv_errors

[1] 24.20520 19.18924 19.30662 19.33799 18.87911 19.02103 18.89609 19.71201
 [9] 18.95140 19.50196